### 1. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from matplotlib import pyplot as plt
from matplotlib import gridspec
import ipywidgets as widgets
from ipywidgets import fixed
from geopy.geocoders import Nominatim
import geopy.geocoders
import certifi
import ssl


### 2. Loading dataset

In [2]:
ds = pd.read_csv('Datasets/kc_house_data.csv')
df = ds

### 3. Business Problems


1. Qual a média do preço de compra dos imóveis por “Nível”?
- Nível 0 -> Preço entre R$ 0 e R$ 321.950
- Nível 1 -> Preço entre R$ 321.950 e R$ 450.000
- Nível 2 -> Preço entre R$ 450.000 e R$ 645.000
- Nível 3 -> Acima de R$ 645.000
2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
- Size 0 -> Tamanho entre 0 e 1427 sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e 2550 sqft
- Size 3 -> Tamanho acima de 2550 sqft
3. Adicione as seguinte informações ao conjunto de dados original:
- Place ID: Identificação da localização
- OSM Type: Open Street Map type
- Country: Nome do País
- Country Code: Código do País
4. Adicione os seguinte filtros no Mapa:
- Tamanho mínimo da área da sala de estar.
- Número mínimo de banheiros.
- Valor Máximo do Preço.
- Tamanho máximo da área do porão.
- Filtro das Condições do Imóvel.
- Filtro por Ano de Construção.
5. Adicione os seguinte filtros no Dashboard:
- Filtro por data disponível para compra.
- Filtro por ano de renovação.
- Filtro se possui vista para a água ou não.


### 5. Solution

In [3]:
# 1. Qual a média do preço de compra dos imóveis por “Nível”?
#     - Nível 0 -> Preço entre R$ 0 e R$321.950
#     - Nível 1 -> Preço entre R$ 321.950 e R$ 450.000
#     - Nível 2 -> Preço entre R$ 450.000 e R$ 645.000
#     - Nível 3 -> Acima de R$ 645.000

# define level of prices
for i in range( len( df ) ):
    if df.loc[i, 'price'] <= 321950:
        df.loc[i, 'level'] = 0
    elif ( df.loc[i,'price'] > 321950 ) & ( df.loc[i,'price'] <= 450000 ):
        df.loc[i, 'level'] = 1
    elif ( df.loc[i,'price'] > 450000 ) & ( df.loc[i,'price'] <= 645000 ):
        df.loc[i, 'level'] = 2
    else:
        df.loc[i, 'level'] = 3

In [4]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,level
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,0.0
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,2.0
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,770,0,1933,0,98028,47.7379,-122.233,2720,8062,0.0
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,2.0
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,2.0


In [5]:
#mean of prices grouped by property level
df[['price', 'level']].groupby('level').mean().reset_index()

,level,price
0,0.0,251557.649889
1,1.0,385688.681502
2,2.0,539730.960007
3,3.0,987540.224456


In [6]:
# 2. Qual a média do tamanho da sala de estar dos imóveis por “Size” ?
# - Size 0 -> Tamanho entre 0 e 1427 sqft
# - Size 1 -> Tamanho entre 1427 e 1910 sqft
# - Size 2 -> Tamanho entre 1910 e 2550 sqft
# - Size 3 -> Tamanho acima de 2550 sqft

# define sizes
for i in range( len( df ) ):
    if df.loc[i, 'sqft_living'] <= 1427:
        df.loc[i, 'size'] = 0
    elif ( df.loc[i,'sqft_living'] > 1427 ) & ( df.loc[i,'sqft_living'] <= 1910 ):
        df.loc[i, 'size'] = 1
    elif ( df.loc[i,'sqft_living'] > 1910 ) & ( df.loc[i,'sqft_living'] <= 2550 ):
        df.loc[i, 'size'] = 2
    else:
        df.loc[i, 'size'] = 3

In [7]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,level,size
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,0,1955,0,98178,47.5112,-122.257,1340,5650,0.0,0.0
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,400,1951,1991,98125,47.7210,-122.319,1690,7639,2.0,3.0
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,0,1933,0,98028,47.7379,-122.233,2720,8062,0.0,0.0
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,910,1965,0,98136,47.5208,-122.393,1360,5000,2.0,2.0
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,0,1987,0,98074,47.6168,-122.045,1800,7503,2.0,1.0


In [8]:
#mean of living room area grouped by property size
df[['sqft_living', 'size']].groupby('size').mean().reset_index()

,size,sqft_living
0,0.0,1123.832531
1,1.0,1664.962334
2,2.0,2211.792786
3,3.0,3329.607016


In [9]:
# 3. Adicione as seguinte informações ao conjunto de dados original:
# - Place ID: Identificação da localização
# - OSM Type: Open Street Map type
# - Country: Nome do País
# - Country Code: Código do País

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE
geopy.geocoders.options.default_ssl_context = ctx

geolocator = Nominatim( scheme='https', user_agent='geoapiExercises')

#API response test
response = geolocator.reverse( '47.5112,-122.257' )
print(response.raw['place_id'])
print(response.raw['osm_type'])
print(response.raw['address']['country'])
print(response.raw['address']['country_code'])


161022547
way
United States
us


In [10]:
#create blank columns
df['place_id'] = 'NA'
df['osm_type'] = 'NA'
df['country'] = 'NA'
df['country_code'] = 'NA'
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,lat,long,sqft_living15,sqft_lot15,level,size,place_id,osm_type,country,country_code
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,47.5112,-122.257,1340,5650,0.0,0.0,NA,NA,NA,NA
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,47.7210,-122.319,1690,7639,2.0,3.0,NA,NA,NA,NA
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,47.7379,-122.233,2720,8062,0.0,0.0,NA,NA,NA,NA
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,47.5208,-122.393,1360,5000,2.0,2.0,NA,NA,NA,NA
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,47.6168,-122.045,1800,7503,2.0,1.0,NA,NA,NA,NA


In [11]:
newatt = ['place_id', 'osm_type', 'country', 'country_code']

for i in range (5): #testing loop for 5 lines

    #defining latitude and longitude
      
    latlong = str(df.loc[i, 'lat']) + ',' + str(df.loc[i, 'long'])

    #API request
    response = geolocator.reverse( latlong )

    for att in newatt:
        if att in response.raw['address']:
            df.loc[i, att] = response.raw['address'][att]
        elif att in response.raw:
            df.loc[i, att] = response.raw[att]

In [12]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,lat,long,sqft_living15,sqft_lot15,level,size,place_id,osm_type,country,country_code
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,47.5112,-122.257,1340,5650,0.0,0.0,161022547,way,United States,us
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,47.7210,-122.319,1690,7639,2.0,3.0,160016678,way,United States,us
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,47.7379,-122.233,2720,8062,0.0,0.0,74951055,node,United States,us
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,47.5208,-122.393,1360,5000,2.0,2.0,156726151,way,United States,us
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,47.6168,-122.045,1800,7503,2.0,1.0,296045104,way,United States,us


In [13]:
#Apply multithread for faster solution

import time
from multiprocessing import Pool

In [14]:
df['latlong'] = df[['lat', 'long']].apply( lambda x: str(x['lat']) + ',' + str(x['long']), axis=1)

In [15]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,long,sqft_living15,sqft_lot15,level,size,place_id,osm_type,country,country_code,latlong
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,-122.257,1340,5650,0.0,0.0,161022547,way,United States,us,"47.5112,-122.257"
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,-122.319,1690,7639,2.0,3.0,160016678,way,United States,us,"47.721,-122.319"
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,-122.233,2720,8062,0.0,0.0,74951055,node,United States,us,"47.7379,-122.233"
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,-122.393,1360,5000,2.0,2.0,156726151,way,United States,us,"47.5208,-122.393"
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,-122.045,1800,7503,2.0,1.0,296045104,way,United States,us,"47.6168,-122.045"


In [ ]:
#defining external function
#
#THIS FUNCION SHOULD BE SAVED AS A .PY FILE NAMED 'defs'
#
# import time
# import geopy.geocoders
# from geopy.geocoders import Nominatim
# import certifi
# import ssl

# ctx = ssl.create_default_context()
# ctx.check_hostname = False
# ctx.verify_mode = ssl.CERT_NONE
# geopy.geocoders.options.default_ssl_context = ctx

# geolocator = Nominatim( user_agent='geopyExercises')

# def get_data(x):
#     index, row = x
   
#     #API request
#     response = geolocator.reverse( row['latlong'] )
#     address = response.raw['address']

#     place_id = response.raw['place_id'] if 'place_id' in response.raw else 'NA'
#     osm_type = response.raw['osm_type'] if 'osm_type' in response.raw else 'NA'
#     country = address['country'] if 'country' in address else 'NA'
#     country_code = address['country_code'] if 'country_code' in address else 'NA'

#     return place_id, osm_type, country, country_code

In [16]:
import defs

df1 = df[['id', 'latlong']].head()
p = Pool(3)

start = time.process_time()
df1[['place_id', 'osm_type', 'country', 'country_code']] = p.map( defs.get_data, df1.iterrows())
end = time.process_time()

print('Time Elapsed: {}', end - start)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=47.5208&lon=-122.393&format=json&addressdetails=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000200DC2F4A90>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))

In [ ]:
df1.head()

In [ ]:
# 4. Adicione os seguinte filtros no Mapa:
# - Tamanho mínimo da área da sala de estar.
# - Número mínimo de banheiros.
# - Valor Máximo do Preço.
# - Tamanho máximo da área do porão.
# - Filtro das Condições do Imóvel.
# - Filtro por Ano de Construção.

In [ ]:
df['is_waterfront'] = df['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no')

In [ ]:
price_limit = widgets.IntSlider(value=540000,
                                     min=75000,
                                     max=7700000,
                                     step=1,
                                     description='Price Limit',
                                     disable=False,
                                     style={'description_wifth': 'initial'})           


In [ ]:
def update_map( df, limit ):
    houses = df[(df['price'] <= limit)] [['id', 'lat', 'long', 'price', 'level']]
    mapa = px.scatter_mapbox( houses, lat='lat', lon='long', color= 'level', size='price', hover_name='id', hover_data=['price'], color_discrete_sequence=['darkgreen'], zoom=9, height=300,)
    mapa.update_layout( mapbox_style='open-street-map')
    mapa.update_layout( height=600, margin={'r':0, 't':0,'l':0,'b':0 })
    mapa.show()

In [ ]:
widgets.interactive( update_map, df=fixed(df), limit=price_limit )

In [ ]:
# Prepare Data
data =df
data['is_waterfront'] = data['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no' )

# define level of prices
data['level'] = data['price'].apply( lambda x: 0 if x <= 321950 else 
                                           1 if ( x > 321950 ) & ( x <= 450000) else 
                                           2 if ( x > 450000 ) & ( x <= 645000 ) else 3 )
data['level'] = data['level'].astype( int )

# Iteractive buttons
price_limit = widgets.IntSlider(
    value = 540000,
    min = 75000,
    max = 7700000,
    step = 1,
    description='Maximun Price',
    disable=False,
    style={'description_width': 'initial'}
)

waterfront_bar = widgets.Dropdown(
    options = data['is_waterfront'].unique().tolist(),
    value = 'yes',
    description = 'Water View',
    disable=False
)

livingroom_limit = widgets.IntSlider(
    value = int( data['sqft_living'].mean() ), 
    min = data['sqft_living'].min(),
    max = data['sqft_living'].max(),
    step = 1,
    description='Minimum Living Room Size',
    disable=False,
    style={'description_width': 'initial'}
)

bathroom_limit = widgets.IntSlider(
    value = int( data['bathrooms'].mean() ), 
    min = data['bathrooms'].min(),
    max = data['bathrooms'].max(),
    step = 1,
    description='Minimum Bathroom Number',
    disable=False,
    style={'description_width': 'initial'}
)

basement_limit = widgets.IntSlider(
    value = int( data['sqft_basement'].mean() ), 
    min = data['sqft_basement'].min(),
    max = data['sqft_basement'].max(),
    step = 1,
    description='Minimum Basement Size',
    disable=False,
    style={'description_width': 'initial'}
)

condition_limit = widgets.IntSlider(
    value = int( data['condition'].mean() ), 
    min = data['condition'].min(),
    max = data['condition'].max(),
    step = 1,
    description='Minimum condition',
    disable=False,
    style={'description_width': 'initial'}
)

year_limit = widgets.IntSlider(
    value = int( data['yr_built'].mean() ), 
    min = data['yr_built'].min(),
    max = data['yr_built'].max(),
    step = 1,
    description='Year Built',
    disable=False,
    style={'description_width': 'initial'}
)


def update_map( df, waterfront, limit, livingroom_limit, bathroom_limit, basement_limit, condition_limit, year_limit ):
    houses = df[(df['price'] <= limit) & 
                (df['is_waterfront'] == waterfront) &
                (df['sqft_living'] >= livingroom_limit) &
                (df['bathrooms'] >= bathroom_limit) &
                (df['sqft_basement'] >= basement_limit) &
                (df['condition'] >= condition_limit) &
                (df['yr_built'] >= year_limit )][['id', 'lat', 'long', 'price', 'level']]
    
    fig = px.scatter_mapbox( houses, 
                             lat="lat", 
                             lon="long", 
                             size="price",
                             color="level",
                             color_continuous_scale=px.colors.cyclical.IceFire, 
                             size_max=15, 
                             zoom=10)

    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(height=600, margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [ ]:
fig = px.scatter_mapbox( data, 
                         lat="lat", 
                         lon="long", 
                         size="price",
                         color="level",
                         color_continuous_scale=px.colors.cyclical.IceFire, 
                         size_max=15, 
                         zoom=10)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(height=600, margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
interactive_plot = widgets.interactive( update_map, 
                    df=fixed(data),
                     waterfront=waterfront_bar, 
                     limit=price_limit, 
                     livingroom_limit=livingroom_limit,
                     bathroom_limit=bathroom_limit,
                     basement_limit=basement_limit,
                     condition_limit=condition_limit,
                     year_limit=year_limit
                    )

In [ ]:
interactive_plot

In [ ]:
# 5. Adicione os seguinte filtros no Dashboard:
# - Filtro por data disponível para compra.
# - Filtro por ano de renovação.
# - Filtro se possui vista para a água ou não.

import ipywidgets as widgets
from ipywidgets import fixed
from matplotlib import gridspec
from matplotlib import pyplot as plt
import pandas as pd

df = pd.read_csv('Datasets/kc_house_data.csv')

In [ ]:
df['renovation_year'] = pd.to_datetime(df['yr_renovated']).dt.strftime( '%Y')
df['date'] = pd.to_datetime(df['date']).dt.strftime( '%Y-%m-%d')
df.head()

In [ ]:
# Change data format
data['year'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y' )
data['date'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y-%m-%d' )
data['year_week'] = pd.to_datetime( data['date'] ).dt.strftime( '%Y-%U' )

# ----------------------------
# Filtering
# ----------------------------
# Widget to control data
date_limit = widgets.SelectionSlider( 
    options = data['date'].sort_values().unique().tolist(),
    value='2014-12-01',
    description='Max Available Date',
    disable=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True
)

# Max Year Renovated
year_limit = widgets.SelectionSlider( 
    options = data['yr_renovated'].sort_values().unique().tolist(),
    value=2000,
    description='Max Year',
    disable=False,
    continuous_update=False,
    orientation='horizontal',
    style={'description_width': 'initial'},
    readout=True
)

# Waterfront
waterfront_limit = widgets.Checkbox(
    value=False,
    description='Waterfront?',
    disabled=False,
    indent=False
)

def update_map( data, date_limit, year_limit, waterfront_limit ):
    
    # Filter data
    df = data[(data['date'] <= date_limit) & 
              (data['yr_renovated'] >= year_limit) &
              (data['waterfront'] == waterfront_limit)].copy()

    fig = plt.figure( figsize=(24, 12) )
    specs = gridspec.GridSpec( ncols=2, nrows=2, figure=fig )

    ax1 = fig.add_subplot( specs[0, :] ) # First Row
    ax2 = fig.add_subplot( specs[1, 0] ) # First Row First Column
    ax3 = fig.add_subplot( specs[1, 1] ) # Second Row First Column

    by_year = df[['id', 'year']].groupby( 'year' ).sum().reset_index()
    ax1.bar( by_year['year'], by_year['id'] )

    # Frist Graph
    by_day = df[['id', 'date']].groupby( 'date' ).mean().reset_index()
    ax2.plot( by_day['date'], by_day['id'] )
    ax2.set_title( "Title: Avg Price by Day" )
    
    df['year_week'] = pd.to_datetime( df['date'] ).dt.strftime( '%Y-%U' )
    by_week_of_year = df[['id', 'year_week']].groupby( 'year_week' ).mean().reset_index()
    ax3.bar( by_week_of_year['year_week'], by_week_of_year['id'] )
    ax3.set_title( "Title: Avg Price by Week of Year" )
    plt.xticks( rotation=60);

In [ ]:
widgets.interactive( update_map, 
                     data = fixed( data ), 
                     date_limit=date_limit,
                     year_limit=year_limit,
                     waterfront_limit=waterfront_limit )